In [ ]:
# import cv2
# import numpy as np
# import os

# def load_images_from_folder(folder):
#     images = []
#     labels = []
#     for label, subfolder in enumerate(os.listdir(folder)):
#         subfolder_path = os.path.join(folder, subfolder)
#         for filename in os.listdir(subfolder_path):
#             img = cv2.imread(os.path.join(subfolder_path, filename), cv2.IMREAD_GRAYSCALE)
#             if img is not None:
#                 img = cv2.resize(img, (64, 64))  # Resize to 64x64
#                 images.append(img.flatten())  # Flatten the image
#                 labels.append(label)
#     return np.array(images), np.array(labels)

# # Load your dataset
# X, y = load_images_from_folder('path_to_your_dataset')


In [ ]:
from skimage.feature import hog

def load_images_with_hog(folder):
    images = []
    labels = []
    for label, subfolder in enumerate(os.listdir(folder)):
        subfolder_path = os.path.join(folder, subfolder)
        for filename in os.listdir(subfolder_path):
            img = cv2.imread(os.path.join(subfolder_path, filename), cv2.IMREAD_GRAYSCALE)
            if img is not None:
                img = cv2.resize(img, (64, 64))  # Resize to 64x64
                fd, _ = hog(img, orientations=8, pixels_per_cell=(8, 8),
                            cells_per_block=(1, 1), visualize=True, multichannel=False)
                images.append(fd)  # Use HOG features
                labels.append(label)
    return np.array(images), np.array(labels)

# Load your dataset with HOG features
X, y = load_images_with_hog('path_to_your_dataset')


In [ ]:
# from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
class SVM:
    def __init__(self, learning_rate=0.001, lambda_param=0.01, n_iters=1000):
        self.lr = learning_rate
        self.lambda_param = lambda_param
        self.n_iters = n_iters
        self.w = None
        self.b = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        y_ = np.where(y <= 0, -1, 1)
        
        self.w = np.zeros(n_features)
        self.b = 0

        for _ in range(self.n_iters):
            for idx, x_i in enumerate(X):
                condition = y_[idx] * (np.dot(x_i, self.w) - self.b) >= 1
                if condition:
                    self.w -= self.lr * (2 * self.lambda_param * self.w)
                else:
                    self.w -= self.lr * (2 * self.lambda_param * self.w - np.dot(x_i, y_[idx]))
                    self.b -= self.lr * y_[idx]
    
    def predict(self, X):
        approx = np.dot(X, self.w) - self.b
        return np.sign(approx)


In [ ]:
def train_one_vs_rest_svm(X_train, y_train):
    svms = []
    for label in np.unique(y_train):
        y_train_binary = np.where(y_train == label, 1, -1)
        svm = SVM()
        svm.fit(X_train, y_train_binary)
        svms.append(svm)
    return svms

def predict_one_vs_rest(svms, X):
    predictions = np.array([svm.predict(X) for svm in svms])
    return np.argmax(predictions, axis=0)

# Train
svms = train_one_vs_rest_svm(X_train, y_train)

# Predict
y_pred = predict_one_vs_rest(svms, X_test)


In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
